# Test system for function development

In [1]:
# DO NOT MODIFY BELOW

# File Handling
import os
import pickle

# Array Handling
import numpy as np
import pandas as pd

# Make Plotting Interactive
from ipywidgets import interact, fixed

# Wrapper for xraylarch (larch)
os.chdir('..')
pwd = os.getcwd()
os.chdir('./catxas')
import general as fcts
import xas as xfcts
import plot as pfcts
import process
import experiment as exp

# Plot Handling
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

%matplotlib inline

In [ ]:
# Where is the data?

# Directory Containing Experiment:
pickle_dir = r'D:\UU QEXAFS Workup\Nina - BF31\QEXAFS Python Workup Testing 20220922'

# Name of Experiment (must include extension '.pickle')
pickle_name = 'BF31-Fe-All.pickle'


In [ ]:
# DO NOT MODIFY BELOW

pickle_path = os.path.join(pickle_dir,pickle_name)
pickle_in = open(pickle_path,"rb")
my_exp = pickle.load(pickle_in)
pickle_in.close()

# Method for Averaging (merging) Spectra


In [ ]:
# List all Spectra in experiment

spectra_list = list(my_exp.spectra.keys())

spectra_list

In [8]:
# Plot first and last spectra

x1 = np.linspace(0, 2, 50)
y1 = np.sin(np.pi*x1)

x2 = np.linspace(0, 2, 50)
y2 = np.cos(np.pi*x2)

xy_list = [[x1, y1], [x2, y2]]

In [3]:
# Find largest difference in spectra
emin = 0
emax = 2


vline_pos = (emin, emax, 0.1) 
axis_label = ['Photon Energy (eV)','Norm mux'] 
size = [12, 10]

interact(pfcts.plot_XYs_Vline, 
         xy_list = fixed(xy_list), 
         vline_pos = vline_pos, 
         emin = fixed(emin), 
         emax = fixed(emax), 
         axis_label = fixed(axis_label), 
         size = fixed(size))


interactive(children=(FloatSlider(value=1.0, description='vline_pos', max=2.0), Output()), _dom_classes=('widg…

<function plot.plot_XYs_Vline(xy_list, vline_pos, emin, emax, axis_label=['Photon Energy (eV)', 'Norm mux'], size=[12, 10])>

In [ ]:
# value from plot above
energy_pt = 7125

In [ ]:
# Convert spectra list postiion to integer in list
spectra_list_number = list(range(len(spectra_list)))

# get flat Norm(mux) value closest to largest energy variation

spectra_mu = []

for key in my_exp.spectra.keys():
    
    # find closest energy point
    idx = fcts.find_nearest(my_exp.spectra[key]['Absorption Spectra']['mu Sample'].energy)
    
    #get flattened mu value and append to spectra_mu list
    spectra_mu.append(my_exp.spectra[key]['Absorption Spectra']['mu Sample'].flat[idx])

In [ ]:
# track mu with spectra number
emin = 0
emax = len(spectra_list)


vline_pos = (emin, emax, 1) 
axis_label = ['Spectra Number in List','Norm mux'] 
size = [12, 10]

interact(plot_scans_Vline, 
         xy_list = fixed(xy_list), 
         vline_pos = vline_pos, 
         emin = fixed(emin), 
         emax = fixed(emax), 
         axis_label = fixed(axis_label), 
         size = fixed(size))

In [ ]:
# Value from above plot

steaty_state_starts = 50

# Downselect spectra list for merging

merge_spectra_list = specrta_list[steaty_state_starts:]

In [ ]:
# Create Merged Spectra
merged_spectra = merge_spectra(my_exp, merge_spectra_list)

In [ ]:
# Save Merge Spectra + Process Params

data = {
    'Energy': merged_spectra['Absorption Spectra']['mu Sample'].energy,
    'Sample mu': merged_spectra['Absorption Spectra']['mu Sample'].mu,
    'Ref mu': merged_spectra['Absorption Spectra']['mu Reference'].mu,
}

df = pd.Dataframe(data)

output_dir = r'path here'

merge_fname = r'filename #1 here'

process_fname = r'filanme #2 here'

In [ ]:
# Save Merge Spectra
df.to_csv(os.path.join(output_dir, merge_fname + '.csv'), sep=',', na_rep='', header=True, index=False)

# Save Process Params
merged_spectra.to_csv(os.path.join(output_dir, process_fname + '.csv'), sep=',', na_rep='', header=True, index=False)